In [ ]:
domain = "docs.sillytavern.app"
docs_url = "https://docs.sillytavern.app/"
!wget -e robots=off --recursive --mirror --no-clobber --page-requisites --html-extension --convert-links --restrict-file-names=windows --domains {domain} --no-parent {docs_url}

In [ ]:
%pip install -q langchain langchain_openai python-dotenv portkey-ai pyyaml html5lib

In [ ]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

dotenv.load_dotenv()


def portkey_llm(model="openai/gpt-4o-mini", temperature=0.5):
    PORTKEY_API_KEY = os.getenv("PORTKEY_API_KEY")
    OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")
    OPEN_ROUTER_URL = os.getenv("OPEN_ROUTER_URL")

    headers = createHeaders(provider="openrouter", api_key=PORTKEY_API_KEY)
    # base_url = "http://localhost:8787/v1"
    # base_url = OPEN_ROUTER_URL
    base_url = PORTKEY_GATEWAY_URL
    # print(headers)

    chat = ChatOpenAI(model=model,
                      api_key=OPEN_ROUTER_API_KEY,
                      base_url=base_url,
                      default_headers=headers,
                      temperature=temperature)
    return chat

In [ ]:
import re
import yaml
from bs4 import BeautifulSoup

file_path = './docs.sillytavern.app/index.html'
file_path1 = './docs.sillytavern.app/index.html.bak'

with open(file_path1, 'r', encoding='utf-8') as file:
    content = file.read()


# 
# content = '''
# <div>a <b>b</b> c <b>d</b>e</div>
# '''

# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(content, 'html.parser')

# 存储元素信息的数组
elements_info = []
index = 0

# 遍历所有元素并获取信息
def get_element_info(element):
    global index
    # 检查节点是否为Tag对象，并且不在排除的标签列表中
    if element.name and element.name not in ['script', 'style', 'noscript', 'head', 'link']:
        # 获取子节点中的文本内容
        text = element.get_text(strip=True)
        if re.search(r'\w', text):
            index += 1
            elements_info.append({
                'index': index,
                'tag': element.name,
                'text': text,
            })
    
    # 递归遍历子元素
    for child in element.children:
        if child.name:  # 确保子节点也是Tag对象
            get_element_info(child)

# 获取所有元素的信息
get_element_info(soup)

# # 只提取 index 和 text
filtered_texts = [{"index": item["index"], "text": item["text"]} for item in elements_info]

# 转换为 YAML 格式
yaml_data = yaml.dump(filtered_texts, allow_unicode=True)

print(yaml_data)



In [ ]:
elements_info

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_template = """
You will be given a YAML formatted input containing entries with "id" and "{imt_source_field}" fields. 

For each entry in the YAML, translate the contents of the "{imt_source_field}" field into {to}. Write the translation back into the "{{imt_source_field}}" field for that entry.

Here is an example of the expected format:

<example>
Input:
  - id: 1
    {imt_source_field}: Source
Output:
  - id: 1
    {imt_trans_field}: Translation
</example>

Please return the translated YAML directly without wrapping <yaml> tag or include any additional information.
"""

user_template = """
Here is the input:

<yaml>
{{yaml}}
</yaml>
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

parser = StrOutputParser()
chain = prompt_template | portkey_llm() | parser
response = chain.invoke({"text": yaml_data, "to": "chinese", "imt_source_field": "text", "imt_trans_field": "translation"})


In [ ]:

translated_texts = yaml.load(response, Loader=yaml.FullLoader)
# 将 source_texts 和 translated_texts 通过index 合并
for source_text in elements_info:
    for translated_text in translated_texts:
        if source_text["index"] == translated_text["index"]:
            source_text["translated"] = translated_text["translation"]

elements_info

In [ ]:
elements_info

In [ ]:
content = '''
<html>
  <body>
    <ul>
        <li>Mobile-friendly interface</li>
        <li>Multiple backend API connectivity (<a href="https://github.com/KoboldAI/KoboldAI-Client">KoboldAI</a> abc, <a href="https://github.com/LostRuins/koboldcpp">KoboldCpp</a>, <a href="https://horde.koboldai.net/">AI Horde</a>, <a href="https://github.com/LostRuins/koboldcpp">NovelAI</a>, <a href="https://github.com/oobabooga/text-generation-webui">Oobabooga's TextGen WebUI</a>, <a href="https://chat.openai.com/">OpenAI</a>, <a href="https://windowai.io">WindowAI</a>, <a href="https://openrouter.ai/">OpenRouter</a>, <a href="https://github.com/theroyallab/tabbyAPI">TabbyAPI</a>, and many more...). See <a href="usage/api-connections/index.html">API connections</a>.</li>
        <li>Visual Novel-like Waifu Mode</li>
        <li>Horde Stable Diffusion generation</li>
        <li>TTS support (ElevenLabs, Silero, etc.)</li>
  </body>
</html>
'''

# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(content, 'html.parser')


# 获取元素的 CSS 选择器
def xpath_to_css(xpath) -> str:
    # 去掉前导的斜杠
    if xpath.startswith('/'):
        xpath = xpath[1:]
    
    if 'comment()' in xpath:
        return None
        
    selectors = []
    for part in xpath.split('/'):
        if '[' in part:
            # 提取标签名和索引
            tag, index = re.match(r'([a-zA-Z0-9_-]+)\[(\d+)\]', part).groups()
            selectors.append(f'{tag}:nth-of-type({index})')
        else:
            selectors.append(part)
    return ' > '.join(selectors)


for element_info in elements_info:

    if element_info["text"] == "Powered by":
        continue

    xpath = element_info["xpath"]
    translated_text = element_info["translated"]

    # 将XPath转换为BeautifulSoup选择器
    selectors = xpath_to_css(xpath)
    if not selectors:
        continue

    # 查找并更新元素的文本内容
    element = soup.select_one(selectors.strip())
    if element:
        element.string = translated_text

# 将修改后的HTML内容写回文件
# with open(file_path, 'w', encoding='utf-8') as file:
#     file.write(str(soup))

print(str(soup))

In [ ]:
file_path = './docs.sillytavern.app/index.html'
file_path1 = './docs.sillytavern.app/index.html.bak'

with open(file_path1, 'r', encoding='utf-8') as file:
    content = file.read()

# content = '''
# <html>
#   <body>
#     <div class="content">
#       <p id="para2">This is another paragraph.</p>
#     </div>
#     <div id="para3">abc<b>123</b></div>
#     <span>Some text here</span>
#     <!-- This is a comment and should be skipped -->
#   </body>
# </html>
# '''

# 使用BeautifulSoup解析HTML内容
soup = BeautifulSoup(content, 'html.parser')

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(str(soup))

In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag
import re

content = '''
<div>abc<div>123</div>456</div>
<div>
<div>def<a>111</a>2342</div>
</div>
<ul>
<li>release -<span class="docs-emoji">&#x1F31F;</span> <strong>Recommended for most users.</strong> This is the most stable and recommended branch, updated only when major releases are pushed. It&#x27;s suitable for the majority of users.</li>
<li>staging - <span class="docs-emoji">&#x26A0;&#xFE0F;</span> <strong>Not recommended for casual use.</strong> This branch has the latest features, but be cautious as it may break at any time. Only for power users and enthusiasts.</li>
</ul>
'''

# 使用 html5lib 解析 HTML 文档
soup = BeautifulSoup(content, 'html.parser')
elements_info = []

def get_text_by_tag(tag):
    if isinstance(tag, NavigableString):
        if re.search(r'\w', tag):
            elements_info.append(str(tag))
        return

    if all(isinstance(child, NavigableString) for child in tag.contents):
        elements_info.append(tag.get_text())
        return

    for child in tag.children:
        get_text_by_tag(child)

# 获取文档中所有顶级标签的文本
for tag in soup.find_all(recursive=False):
    get_text_by_tag(tag)

# 打印结果
print('\n@'.join(elements_info))


In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag
import re

content = '''
<div>abc<div>123</div>456</div>
<div>
<div>def<a>111</a>2342</div>
</div>
<ul>
<li>release -<span class="docs-emoji">&#x1F31F;</span> <strong>Recommended for most users.</strong> This is the most stable and recommended branch, updated only when major releases are pushed. It&#x27;s suitable for the majority of users.</li>
<li>staging - <span class="docs-emoji">&#x26A0;&#xFE0F;</span> <strong>Not recommended for casual use.</strong> This branch has the latest features, but be cautious as it may break at any time. Only for power users and enthusiasts.</li>
</ul>
'''

# 使用 html5lib 解析 HTML 文档
soup = BeautifulSoup(content, 'html.parser')
elements_info = []

def get_text_by_tag(tag):
    if isinstance(tag, NavigableString):
        if re.search(r'\w', tag):
            elements_info.append(str(tag))
        return

    if all(isinstance(child, NavigableString) for child in tag.contents):
        elements_info.append(tag.get_text())
        return

    for child in tag.children:
        get_text_by_tag(child)

# 获取文档中所有顶级标签的文本
for tag in soup.find_all(recursive=False):
    get_text_by_tag(tag)

# 打印结果
print('\n@'.join(elements_info))

In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag
import re

file_path = './docs.sillytavern.app/index.html'
file_path1 = './docs.sillytavern.app/index.html.bak'

with open(file_path1, 'r', encoding='utf-8') as file:
    content = file.read()


# 使用 html5lib 解析 HTML 文档
soup = BeautifulSoup(content, 'html.parser')
elements_info = []

def get_text_without_parent_tag(tag):
    texts = []
    for child in tag.children:
        texts.append(str(child))
        
    text = ''.join(texts)
    
    return {
        'tag': tag.name,
        'text': text
    }

def get_text_by_tag(tag):

    if isinstance(tag, NavigableString):
        if re.search(r'\w', tag):
            elements_info.append(str(tag).strip())
        
        return

    # 单文本节点, 有文本没有子节点
    if len(tag.contents)>0 and is_string(tag.contents):
        elements_info.append(get_text_without_parent_tag(tag))
        return
        
    # 如果是包裹元素的文本节点，没有特定标签，直接返回文本内容
    if is_string_tag(tag.contents):
        elements_info.append(get_text_without_parent_tag(tag))
        return

    # 如果是包裹元素的文本节点，有特定标签，直接返回文本内容
    # todo

    # 如果有子节点，递归遍历子节点
    for child in tag.children:
        get_text_by_tag(child)
        
def is_string(contents):
    return isinstance(contents[0], NavigableString) and not any(isinstance(tag, Tag) for tag in contents)


def is_string_tag(contents):
    is_navigable_string = False
    is_tag = False
    
    for tag in contents:
        if isinstance(tag, NavigableString) and re.search(r'\w', tag):
            is_navigable_string = True
        
        if isinstance(tag, Tag) and tag.name not in ['div', 'ul']:
            is_tag = True
    
    return is_navigable_string and is_tag


all_texts = []

for tag in soup.find_all():
    get_text_by_tag(tag)

for element_info in elements_info:
    print(element_info)

In [ ]:
from bs4 import BeautifulSoup
import re

def extract_html_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    elements_info = []

    for element in soup.find_all(True):  # True finds all tags
        tag = element.name
        text = ''.join([str(c) for c in element.contents if isinstance(c, str)]).strip()
        
        if tag in ['script', 'style', 'noscript', 'head', 'link']:
            continue

        # Skip elements with empty text
        if not text or not re.search(r'\w', text):
            continue

        # Generate CSS selector
        path = []
        current = element
        while current:
            sibling_count = 0
            if current.parent:
                for sibling in current.parent.find_all(current.name, recursive=False):
                    if sibling is current:
                        break
                    sibling_count += 1
                path.append(f"{current.name}:nth-of-type({sibling_count + 1})")
                current = current.parent
            else:
                path.append(current.name)
                break

        css_selector = " > ".join(reversed(path))

        elements_info.append({
            'tag': tag,
            'text': text,
            # 'css_selector': css_selector
        })

    return elements_info


# file_path = './docs.sillytavern.app/index.html'
# file_path1 = './docs.sillytavern.app/index.html.bak'
# 
# with open(file_path1, 'r', encoding='utf-8') as file:
#     content = file.read()

content = """
<li>Multiple backend API connectivity (
<a href="https://github.com/KoboldAI/KoboldAI-Client">KoboldAI</a>, 
<a href="https://github.com/LostRuins/koboldcpp">KoboldCpp</a>, <a href="https://horde.koboldai.net/">AI Horde</a>, <a href="https://github.com/LostRuins/koboldcpp">NovelAI</a>, <a href="https://github.com/oobabooga/text-generation-webui">Oobabooga's TextGen WebUI</a>, <a href="https://chat.openai.com/">OpenAI</a>, <a href="https://windowai.io">WindowAI</a>, <a href="https://openrouter.ai/">OpenRouter</a>, <a href="https://github.com/theroyallab/tabbyAPI">TabbyAPI</a>, and many more...). See <a href="usage/api-connections/index.html">API connections</a>.</li>
<li>Visual Novel-like Waifu Mode</li>
"""
    
elements_info = extract_html_info(content)
for info in elements_info:
    print(info)


In [4]:
from bs4 import BeautifulSoup, NavigableString, Tag
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)

tag = soup.a
tag.string = "111"
type(tag)

bs4.element.Tag

In [ ]:
    def get_text_by_tag(self, tag):

        if isinstance(tag, NavigableString):
            self.tag_list.append(tag)
            return

        if tag.string and re.search(r'\w', tag.string) and tag.contents and not any(isinstance(t, Tag) for t in tag.contents):  # If it's a NavigableString and not a Tag
            self.tag_list.append(tag.string)
            return

        for child in tag.children:
            if isinstance(child, NavigableString):
                 self.tag_list.append(child)
                 return
            elif isinstance(child, Tag):
                self.tag_list.extend(self.get_text_by_tag(child))
                return
